# Lọc dựa trên nội dung (Content Based Filtering)

Thời lượng ước tính: **45** phút

## Mục tiêu

Sau khi hoàn thành lab này bạn sẽ có thể:

-   Tạo một hệ thống gợi ý sử dụng dựa trên nội dung (Content Based Filtering)


Hệ thống đề xuất (Recommendation system) là một tập hợp các thuật toán được sử dụng để đề xuất các mặt hàng cho người dùng dựa trên thông tin lấy từ người dùng. Các hệ thống này đã trở nên phổ biến và có thể thấy nhiều ở các cửa hàng trực tuyến, cơ sở dữ liệu phim ảnh và công cụ tìm việc. Trong notebook này, chúng ta sẽ khám phá các Hệ thống đề xuất dựa trên Nội dung (Content-based recommendation systems) và triển khai phiên bản đơn giản của một hệ thống sử dụng Python và thư viện Pandas.


### Mục lục

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref2">Tiền xử lý (Preprocessing)</a></li>
        <li><a href="#ref3">Lọc dựa trên nội dung (Content-Based Filtering)</a></li>
    </ol>
</div>
<br>


<a id="ref2"></a>

# Tiền xử lý (Preprocessing)


Đầu tiên, chúng ta hãy thực hiện import:


In [ ]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Bây giờ hãy đọc từng tệp vào khung dữ liệu (Dataframe):


In [ ]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

Hãy xóa year khỏi cột **title** bằng cách sử dụng hàm thay thế của pandas và lưu trữ trong cột **year** mới.

In [ ]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

Cùng với đó, hãy cũng chia các giá trị trong cột **Genres** thành **list of Genres** để đơn giản hóa việc sử dụng trong tương lai. Điều này có thể đạt được bằng cách áp dụng hàm split spring của Python trên đúng cột.


In [ ]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

Vì việc giữ các thể loại ở định dạng danh sách không phải là tối ưu đối với kỹ thuật hệ thống đề xuất dựa trên nội dung nên chúng ta sẽ sử dụng kỹ thuật mã hóa One Hot để chuyển đổi list of Genres thành một vectơ trong đó mỗi cột tương ứng với một giá trị khả thi của đặc trưng. Mã hóa này cần thiết để cung cấp dữ liệu phân loại. Trong trường hợp này, chúng ta lưu trữ mọi thể loại khác nhau trong các cột chứa 1 hoặc 0. 1 cho biết phim thuộc thể loại đó và 0 cho biết là không phải. Ta cũng lưu trữ khung dữ liệu này trong một biến khác vì thể loại sẽ không quan trọng đối với hệ thống đề xuất đầu tiên.


In [ ]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

Tiếp theo, hãy xem khung dữ liệu ratings.


In [ ]:
ratings_df.head()

Mỗi hàng trong khung dữ liệu ratings có một id người dùng liên kết với ít nhất 1 phim, xếp hạng và dấu thời gian hiển thị khi họ đánh giá phim đó. Chúng ta không cần cột dấu thời gian, vì vậy hãy bỏ nó để tiết kiệm bộ nhớ.


In [ ]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

<a id="ref3"></a>

# Hệ thống đề xuất dựa trên nội dung (Content-Based recommendation system)


Bây giờ, chúng ta hãy xem cách triển khai hệ thống đề xuất **dựa trên nội dung** hoặc **hệ thông đề xuất dựa trên mặt hàng-mặt hàng tương tự (Item-Item recommendation system)**. Kỹ thuật này cố gắng tìm ra khía cạnh yêu thích của người dùng của một mặt hàng (item) là gì, sau đó đề xuất các mặt hàng thể hiện những khía cạnh đó. Trong trường hợp của chúng ta, hãy cố gắng tìm ra thể loại yêu thích của đầu vào (input) từ các bộ phim và xếp hạng được đưa ra.

Hãy bắt đầu bằng cách tạo input user (đầu vào người dùng) để đề xuất phim:

Lưu ý: Để thêm nhiều phim hơn, chỉ cần tăng số lượng phần tử trong **userInput**. Hãy thêm tùy ý bạn muốn! Chỉ cần đảm bảo viết hoa hoa chữ cái đầu và nếu một bộ phim bắt đầu bằng "The", chẳng hạn như "The Matrix" thì hãy viết nó như sau: 'Matrix, The'.


In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

#### Thêm movieId vào input user

Để hoàn tất đầu vào (input), hãy trích xuất ID của đầu vào phim từ khung dữ liệu movies và thêm chúng vào đó.

Chúng ta có thể đạt được điều này trước tiên bằng cách lọc ra các hàng chứa tiêu đề của đầu vào phim và sau đó hợp nhất tập con này với khung dữ liệu đầu vào. Chúng ta cũng loại bỏ các cột không cần thiết cho đầu vào để tiết kiệm dung lượng bộ nhớ.


In [ ]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

Chúng ta sẽ bắt đầu bằng cách tìm hiểu các tùy chọn của đầu vào, vì vậy hãy lấy tập hợp con các bộ phim mà đầu vào đã xem từ khung dữ liệu chứa các thể loại được xác định bằng các giá trị nhị phân.

In [ ]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

Chúng ta sẽ chỉ cần bảng genre, vì vậy hãy dọn bảng này một chút bằng cách đặt lại chỉ mục và loại bỏ các cột movieId, title, genres và year.


In [ ]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

Bây giờ chúng ta đã sẵn sàng để bắt đầu tìm hiểu các tùy chọn của đầu vào!

Để làm điều này, chúng ta sẽ chuyển từng thể loại (genre) thành trọng số (weights) bằng cách sử dụng các đánh giá của đầu vào và nhân chúng vào bảng genre và sau đó tổng hợp bảng kết quả theo cột. Thao tác này thực ra là một tích vô hướng giữa ma trận và vectơ nên chúng ta có thể thực hiện đơn giản bằng cách gọi hàm "dot" của Pandas.


In [ ]:
inputMovies['rating']

In [ ]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Bây giờ, chúng ta có trọng số cho mọi sở thích của người dùng, là User Profile (Hồ sơ người dùng). Nhờ nó, chúng ta có thể đề xuất các bộ phim đáp ứng sở thích của người dùng.


Hãy bắt đầu bằng cách trích xuất bảng genre từ khung dữ liệu ban đầu:


In [ ]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

In [ ]:
genreTable.shape

Với hồ sơ của đầu vào và danh sách đầy đủ các phim và thể loại, chúng ta sẽ lấy điểm trung bình có trọng số của mọi bộ phim dựa trên hồ sơ đầu vào và đề xuất 20 phim hàng đầu đáp ứng nó nhất.

In [ ]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

In [ ]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

Bây giờ đây là bảng đề xuất!


In [ ]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

### Ưu điểm và nhược điểm của Lọc dựa trên Nội dung

##### Ưu điểm

-   Tìm hiểu sở thích của người dùng
-   Cá nhân hóa cao cho người dùng

##### Nhược điểm

-   Không tính đến suy nghĩ của người khác về mặt hàng, do đó có thể dẫn đến các đề xuất về mặt hàng chất lượng thấp
- Việc trích xuất dữ liệu không phải lúc nào cũng trực quan
- Việc xác định đặc điểm của mặt hàng mà người dùng không thích hoặc thích không phải lúc nào cũng rõ ràng


### Cảm ơn bạn đã hoàn thành lab này!

Nguồn bài Lab: **IBM**